# LangChainの使い方
このノートブックでは、最新の LangChain（v0.3~）を使った各用途別のサンプルを、逐次的に紹介します。

## 1. LLM ラッパー
OpenAI の API 呼び出しを抽象化したモデルラッパーです。

In [ ]:
from langchain_openai import AzureChatOpenAI
import os
from dotenv import load_dotenv

# 環境設定を読込
load_dotenv()

# 生成AIのチャットモデルを作成
llm = AzureChatOpenAI(
    azure_deployment=os.environ.get("AZURE_OPENAI_LLM_DEPLOYMENT_MODEL"),
    api_version=os.environ.get("AZURE_OPENAI_LLM_DEPLOYMENT_API_VERSION"),
    temperature=0.7
)

# モデル実行
print(llm.invoke([{"role": "user", "content": "こんにちは"}]).content)

## 2. プロンプトテンプレートとチェイン

In [ ]:
from langchain_openai import AzureChatOpenAI
from langchain_core.prompts.chat import PromptTemplate
from langchain_core.output_parsers import StrOutputParser


# 1.生成AIモデルを作成
llm = AzureChatOpenAI(
    azure_deployment=os.environ.get("AZURE_OPENAI_LLM_DEPLOYMENT_MODEL"),
    api_version=os.environ.get("AZURE_OPENAI_LLM_DEPLOYMENT_API_VERSION"),
    temperature=0.7
)

# 2.プロンプト雛型を作成
# input_variablesにパラメータを定義する
# templateに雛型のプロンプトを作成し置換パラメータを埋め込む
prompt = PromptTemplate(
    input_variables=["language", "word"],
    template="次の単語を{language}語に翻訳してください：{word}"
)

# 3.チェインを作成
# 指示(Prompt)、モデル(Model)、回答のパーサー(Parser)を組み合わせたチェイン作成
# StrOutputParserは回答を文字列として取得するパーサー
chain = prompt | llm | StrOutputParser()

# チェインを実行
# 1つ目のチェインの指示に埋め込むパラメータを辞書形式で渡す
print(chain.invoke({
    "language": "日本",
    "word": "りんご"
}))

In [ ]:
from langchain_openai import AzureChatOpenAI
from langchain_core.output_parsers import StrOutputParser
from langchain.prompts import PromptTemplate

# 1.生成AIモデルを作成
llm = AzureChatOpenAI(
    azure_deployment=os.environ.get("AZURE_OPENAI_LLM_DEPLOYMENT_MODEL"),
    api_version=os.environ.get("AZURE_OPENAI_LLM_DEPLOYMENT_API_VERSION"),
    temperature=0.7
)

# 2.チェインの作成
## 要約プロンプト
summary_prompt = PromptTemplate(
    input_variables=["text"],
    template="次の単語について解説してください。: {text}"
)
## 要約チェイン
summary_chain = summary_prompt | llm | StrOutputParser()

## 翻訳プロンプト
translate_prompt = PromptTemplate(
    input_variables=["text"],
    template="英語に翻訳してください。: {text}"
)

translate_chain = translate_prompt | llm | StrOutputParser()

# 3.チェインを組み合わせ
# smmaryの結果をtranslate_chainに渡す
chain = summary_chain | translate_chain

# チェインを実行
# 単語を解説してから英語に翻訳
print(chain.invoke("大規模言語モデル"))


チャット(やり取り)をテンプレート化

In [ ]:
from langchain.prompts import (
    HumanMessagePromptTemplate,
    SystemMessagePromptTemplate,
    AIMessagePromptTemplate,
    ChatPromptTemplate
)

# ------------------------------------------------------
# 単一のチャットをテンプレート化する
# ------------------------------------------------------

# システムプロンプトをテンプレート化
system_template = SystemMessagePromptTemplate.from_template(
    "これから提示するテーマに{language}で答えてください。")
print(system_template.format_messages(language="英語"))
# => [HumanMessage(content="これから提示するテーマに英語で答えてください。")]

# 人間からの単一メッセージをテンプレート化
human_template = HumanMessagePromptTemplate.from_template("本日の{theme}について")
print(human_template.format_messages(theme="天気"))
# => [HumanMessage(content="User: 今日は天気は？")]

# AIの単一メッセージをテンプレート化
ai_template = AIMessagePromptTemplate.from_template(
    "本日の{theme}は{answer}となっています。")
print(ai_template.format_messages(theme="天気", answer="晴れ"))
# => [AIMessage(content='本日の天気は晴れとなっています。')]

# ------------------------------------------------------
# 単一のチャット雛型をまとめる。
# ------------------------------------------------------
print(
    ChatPromptTemplate.from_messages([
        system_template,
        human_template,
        ai_template
    ]).format_prompt(
        language="英語",
        theme="天気",
        answer="晴れ"
    ))

# ------------------------------------------------------
# 初めからまとめて作成する
# ------------------------------------------------------
# 複数のチャットをまとめて定義する
chat_template = ChatPromptTemplate.from_messages([
    ("system", "これから提示するテーマに{language}で答えてください。"),
    ("human", "本日の{theme}について"),
    ("ai", "本日の{theme}は{answer}となっています。")
])

print(
    chat_template.format_prompt(
        language="英語",
        theme="天気",
        answer="晴れ"
    ))

チャットからテキスト生成を行う

In [ ]:
from langchain_openai import AzureChatOpenAI
from langchain_core.output_parsers import StrOutputParser
from langchain.prompts import ChatPromptTemplate


# 1.生成AIモデルを作成
llm = AzureChatOpenAI(
    azure_deployment=os.environ.get("AZURE_OPENAI_LLM_DEPLOYMENT_MODEL"),
    api_version=os.environ.get("AZURE_OPENAI_LLM_DEPLOYMENT_API_VERSION"),
    temperature=0.7
)

# 2.チャット雛型(あるいは履歴)を作成
chat_template = ChatPromptTemplate.from_messages([
    ("system", "{theme}に関する話題について答えてください。"),
    ("human", "{theme}の{question}は？"),
    ("ai", "{theme}の{question}は{answer}となります。"),
    ("human", "{additional}")
])

# 3.チェインを組み立て
chain = chat_template | llm | StrOutputParser()

# チェインを実行
chain.invoke({
    "theme": "本日",
    "question": "天気",
    "answer": "雨",
    "additional": "傘は必要か？"
})

## 4. エージェント（Agents:LangGraph版）
LangChain版とLangGraph版があるが、LangGraph版への移行が推奨されているためLangGraph版で解説する

In [ ]:
import os
from langchain_openai import AzureChatOpenAI
from langgraph.prebuilt import create_react_agent
from langchain_core.tools import tool
# from langchain.agents import load_tools
from datetime import datetime,date


# 1.生成AIモデルを作成
llm = AzureChatOpenAI(
    azure_deployment=os.environ.get("AZURE_OPENAI_LLM_DEPLOYMENT_MODEL"),
    api_version=os.environ.get("AZURE_OPENAI_LLM_DEPLOYMENT_API_VERSION"),
    temperature=0.7
)

# 2.ツールの設定
# Note:LangChainには便利なツールが標準でいくつか定義されている。
# ただし、ツールによっては外部サービスのAPIキーが必要になるので調べてみてください。
# serpapiは、Google検索をプログラムで実行してくれるAPIサービスのツールですが、APIキーの取得が必要です。
# tools = load_tools(["serpapi"], llm=llm)

# 自作のツールを定義する
# RAGを構築する際は検索する部分をフレームワークを使うか自作する

@tool
def today_is():
    "今日の日付を返す"
    return  datetime.today().date()

@tool
def fetch_weather(dt:date):
    "指定された日付の天気を返す"
    today = datetime.today().date()
    if dt < today:
        return "雨"
    elif dt == today:
        return "曇りのち晴れ"
    else:
        return "晴れ"

tools = [
    today_is,
    fetch_weather
]

# エージェントを構築する
agent = create_react_agent(model=llm, tools=tools, prompt="You are a helpful assistant")
agent.invoke({
    "messages": [{"role": "user", "content": "今日の日付と天気は？"}]
})


## 5. メモリ管理（LangGraph 永続化）

In [ ]:
from langgraph.checkpoint.memory import MemorySaver
from langchain_openai import ChatOpenAI
from langchain.agents.react.agent import create_react_agent
from langchain import AgentExecutor

memory = MemorySaver()
olapi = ChatOpenAI(model="gpt-4o-mini")
agent_mem = create_react_agent(
    llm=olapi,
    tools=[],
    checkpointer=memory
)
executor_mem = AgentExecutor.from_agent_and_tools(agent=agent_mem, tools=[], verbose=True)

# 会話の例
a1 = executor_mem.invoke({"input": "こんにちは"})
a2 = executor_mem.invoke({"input": "昨日の話を覚えてる？"})
print(a2)

## 6. ドキュメントロード & テキスト分割

In [ ]:
from langchain_community.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

loader = PyPDFLoader("sample.pdf")
docs = loader.load()
splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
chunks = splitter.split_documents(docs)
print(f"チャンク数: {len(chunks)}")

## 7. 埋め込み & ベクターストア

In [ ]:
from langchain_community.embeddings import OpenAIEmbeddings
from langchain_community.vectorstores import FAISS

embeddings = OpenAIEmbeddings()
vectorstore = FAISS.from_documents(chunks, embeddings)
results = vectorstore.similarity_search("LangChain のユースケース", k=3)
for doc in results:
    print(doc.page_content[:200])

## 8. 検索＋回答（Retrieval Chain）

In [ ]:
from langchain.chains import create_retrieval_chain

qa_chain = create_retrieval_chain(
    retriever=vectorstore.as_retriever(),
    llm=llm,
    prompt=prompt
)
answer = qa_chain.invoke({"query": "LangChain のメモリ管理方法は？"})
print(answer)

## 9. コールバック（ストリーミング出力）

In [ ]:
from langchain_core.callbacks.streaming_stdout import StreamingStdOutCallbackHandler
from langchain_openai import ChatOpenAI

handler = StreamingStdOutCallbackHandler()
llm_stream = ChatOpenAI(streaming=True, callbacks=[handler])
_ = llm_stream.invoke([{"role":"user","content":"最新ニュースを教えて"}])